In [134]:
using Catlab
using Catlab.CategoricalAlgebra
using Catlab.Graphs
using Catlab.Graphs.BasicGraphs
using Catlab.Graphics
using Catlab.Graphics.Graphviz
using Catlab.Graphics.GraphvizGraphs
using Catlab.Graphs.PropertyGraphs
using Catlab.CategoricalAlgebra.CSets
using Catlab.CategoricalAlgebra.FinSets, Catlab.CategoricalAlgebra
using DataStructures
using Colors
using Random

In [135]:
draw(g) = to_graphviz(g, node_labels=true)

draw (generic function with 1 method)

In [136]:
struct Preorder
  carrier:: FinSet
  relation :: SortedSet{Pair{Int,Int}}
  """Construct valid preorder by taking reflexive/transitive closure"""
  function Preorder(carrier:: FinSet, rel :: Vector{Pair{Int,Int}})
    for (a,b) ∈ rel
      a ∈ carrier && b ∈ carrier || error("relation element not in carrier set")
    end
    relation = SortedSet(rel)
    for (a, b, c) in Iterators.product(carrier, carrier, carrier)
      if ((a => b) ∈ relation) && ((b => c) ∈ relation)
        push!(relation, a => c) # enforces relation is transitive
      end
    end
    for i in carrier
      push!(relation, i => i) # enforces that relation is reflexive
    end
    return new(carrier, relation)
  end
end

In [137]:
function load_major(path::String)

    file = open(path, "r")

    num_to_class = Dict()
    class_to_num = Dict()

    relations = Vector{Pair{Int, Int}}()

    num_classes = 0
    while !eof(file)
        line = readline(file)
        if length(strip(line, ' ')) != 0
            sp = split(line, ":")

            if length(sp) >= 1
                name = strip(sp[1], ' ')
                num_classes+=1
                num_to_class[num_classes] = string(name)
                class_to_num[string(name)] = num_classes
            end

            if length(sp) >= 2
                reqs_str = strip(sp[2], ['[', ']', ' '])
                reqs = split(reqs_str, ",")
                
                for req in reqs
                    req = strip(req, ' ')
                    if length(req) > 0
                        if !haskey(class_to_num, req)
                            num_classes+=1
                            num_to_class[num_classes] = string(req)
                            class_to_num[string(req)] = num_classes
                        end
                        push!(relations, class_to_num[req] => class_to_num[name])
                    end
                end
            end
        end
    end

    close(file)


    return (Preorder(FinSet(num_classes),relations), num_to_class)
end

load_major (generic function with 1 method)

In [138]:
cs, cs_mappings = load_major("reqs/CS.reqs")
collect(cs.carrier)

35-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
  ⋮
 27
 28
 29
 30
 31
 32
 33
 34
 35

In [139]:
function top_sort(preorder::Preorder, seed::Int = 0)
    vertices = preorder.carrier
    edges = preorder.relation
    Random.seed!(seed)

    l = []
    q = Queue{String}()
    d = Dict{String, Int}()

    for v in vertices
        d[v] = 0
        for (a,b) in edges
            if v == b && a != b
                d[v]+=1
            end
        end
    end


    
    # randomizes the starting order of the queue
    for v in shuffle!(collect(vertices))
        if d[v] == 0
            enqueue!(q, v)
        end
    end


    while !isempty(q)
        v = dequeue!(q)
        push!(l, v)

        for (a,b) in shuffle!(collect(edges))
            if v == a && a != b
                d[b]-=1
                if d[b] == 0
                    enqueue!(q, b)
                end
            end
        end
    end
    
    return l
end

top_sort (generic function with 2 methods)

In [140]:
for class in top_sort(cs)
    println(class)
end

MethodError: MethodError: Cannot `convert` an object of type Int64 to an object of type String
Closest candidates are:
  convert(::Type{String}, !Matched::String) at essentials.jl:210
  convert(::Type{T}, !Matched::T) where T<:AbstractString at strings/basic.jl:231
  convert(::Type{T}, !Matched::AbstractString) where T<:AbstractString at strings/basic.jl:232
  ...

In [141]:
function find_one_offs(preorder::Preorder)
    one_offs = Vector()
    d = Dict()
    for class in collect(preorder.carrier)
        d[class] = true
    end
    for (a,b) in preorder.relation
        if a != b
            d[a] = false
            d[b] = false
        end
    end
    for (class, is_one_off) in d
        if is_one_off
            push!(one_offs, class)
        end
    end
    return Set(one_offs)
end

function generate_schedule(preorder::Preorder, num_semesters::Int=8, seed::Int=0)
    Random.seed!(seed)

    classes = collect(preorder.carrier)
    reqs = preorder.relation

    full_schedule = Vector()

    one_off_classes = find_one_offs(preorder)


    # build dictionary of in counts like in topological sort
    in_counts = Dict()
    for class in classes
        if !(class in one_off_classes)
            in_counts[class] = 0
            for (a,b) in preorder.relation
                if class == b && a != b
                    in_counts[class]+=1
                end
            end
        end
    end


    # schedule classes with prereqs
    for index in 1:num_semesters
        semester_schedule = Vector()

        available_classes = Set(collect(in_counts))

        # find bottoms
        # marking them for deletion
        for (class, in_count) in shuffle!(collect(available_classes))
            if in_count == 0 && length(semester_schedule) < ceil(length(preorder.carrier)/num_semesters)
                push!(semester_schedule, class)
            end
        end

        # delete taken classes
        for class in semester_schedule
            for (a,b) in preorder.relation
                if class == a
                    in_counts[b]-=1
                end
            end
            delete!(in_counts, class)
        end
        push!(full_schedule, semester_schedule)
    end


    # schedule classes with no prereqs
    # prioritizing even credit distribution
    while length(one_off_classes) > 0
        
        # find semester with min number of credits
        min_index = 1
        for (index, semester) in enumerate(full_schedule)
            if length(full_schedule[index]) < length(full_schedule[min_index])
                min_index = index
            end 
        end
        class_to_add = collect(one_off_classes)[1]
        push!(full_schedule[min_index], class_to_add)
        delete!(one_off_classes, class_to_add)
    end

    if length(one_off_classes) > 0 || length(collect(in_counts)) > 0
        throw("Cannot generate schedule!")
    end

    return full_schedule
end

generate_schedule (generic function with 3 methods)

In [156]:
function convert_to_readable(schedule, mapping)
    for (i,semester_schedule) in enumerate(schedule)
        for (j,semester_schedule) in enumerate(semester_schedule)
            schedule[i][j] = mapping[schedule[i][j]]
        end
    end
    return schedule
end
schedule = convert_to_readable(generate_schedule(cs, 8), cs_mappings)

8-element Vector{Any}:
 Any["Interdisciplinary-1", "MAC2311", "Social/Behavioral2", "Social/Behavioral1"]
 Any["MAC2312", "PHY2048/PHY2048L", "Interdisciplinary-2", "STA3032", "COP3502"]
 Any["Interdisciplinary-3", "MAC2313", "MAS3114", "COP3503"]
 Any["PHY2049/PHY2049L", "COT3100", "Interdisciplinary-4", "EGS4034"]
 Any["CDA3101", "COP3530", "Interdisciplinary-5", "Humanities1"]
 Any["COP4020", "CIS4930-2", "CIS4930-4", "COP4600", "CIS4301"]
 Any["CIS4930-5", "CIS4930-3", "CIS4930-1", "COP4533", "CNT4007"]
 Any["CEN3031", "Humanities2", "HumOrSocOrPhy1", "ENC3246"]

In [157]:
schedule = convert_to_readable(generate_schedule(cs, 10), cs_mappings)

10-element Vector{Any}:
 Any["Interdisciplinary-1", "MAC2311", "HumOrSocOrPhy1", "ENC3246"]
 Any["MAC2312", "PHY2048/PHY2048L", "Interdisciplinary-2", "STA3032"]
 Any["MAC2313", "MAS3114", "Interdisciplinary-3", "COP3502"]
 Any["PHY2049/PHY2049L", "COP3503", "Interdisciplinary-4"]
 Any["Interdisciplinary-5", "COT3100", "Social/Behavioral1"]
 Any["CDA3101", "COP3530", "EGS4034"]
 Any["COP4020", "COP4533", "CEN3031", "CIS4930-4"]
 Any["CIS4301", "COP4600", "CIS4930-1", "CIS4930-2"]
 Any["CIS4930-3", "CNT4007", "CIS4930-5"]
 Any["Humanities2", "Social/Behavioral2", "Humanities1"]

In [158]:
schedule = convert_to_readable(generate_schedule(cs, 10), cs_mappings)

10-element Vector{Any}:
 Any["Interdisciplinary-1", "MAC2311", "HumOrSocOrPhy1", "ENC3246"]
 Any["MAC2312", "PHY2048/PHY2048L", "Interdisciplinary-2", "STA3032"]
 Any["MAC2313", "MAS3114", "Interdisciplinary-3", "COP3502"]
 Any["PHY2049/PHY2049L", "COP3503", "Interdisciplinary-4"]
 Any["Interdisciplinary-5", "COT3100", "Social/Behavioral1"]
 Any["CDA3101", "COP3530", "EGS4034"]
 Any["COP4020", "COP4533", "CEN3031", "CIS4930-4"]
 Any["CIS4301", "COP4600", "CIS4930-1", "CIS4930-2"]
 Any["CIS4930-3", "CNT4007", "CIS4930-5"]
 Any["Humanities2", "Social/Behavioral2", "Humanities1"]

In [145]:
function output_plan(schedule, file_path::String)
    file = open(file_path, "w")
    for semester_schedule in schedule
        for (index, class) in enumerate(semester_schedule)
            if index == length(semester_schedule)
                print(file, class)
            else
                print(file, class * ", ")
            end
        end
        println(file, "")
    end
    close(file)
end

function read_plan(file_path::String)
    lines = readlines(file_path)
    schedule = Vector()
    for line in lines
        semester_schedule = Vector()
        classes = split(line, ", ")
        for class in classes
            push!(semester_schedule, class)
        end
        push!(schedule, semester_schedule)
    end
    return schedule
end

read_plan (generic function with 1 method)

In [146]:
schedule = generate_schedule(cs, 8)
output_plan(schedule, "semester_plans/example1.plan")

MethodError: MethodError: no method matching *(::Int64, ::String)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:560
  *(::T, !Matched::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:88
  *(!Matched::Union{AbstractChar, AbstractString}, ::Union{AbstractChar, AbstractString}...) at strings/basic.jl:260
  ...

In [147]:
schedule = read_plan("semester_plans/example1.plan")

Any[]

In [66]:
struct Monotone_map
    domain::Preorder
    codomain::Preorder
    mapping::FinFunction
    function Monotone_map(domain::Preorder, cod::Preorder, mapping::FinFunction)
      ((dom(mapping) == domain.carrier) && (codom(mapping) == cod.carrier)
      ) || error("mapping mismatch")
      return new(domain,cod,mapping)
    end
  end


function is_monotone(mm::Monotone_map)::Bool
    for comp in mm.domain.relation
        mapL = mm.mapping(comp.first)
        mapR = mm.mapping(comp.second)
        mono=false
        
        if(comp.first == comp.second) #reflexive relation
          continue
        end
        if(mapL == mapR) #pre req and class taken in same semeester
          return false;
        end

        for comparison in mm.codomain.relation

            if(mapL == comparison.first && mapR == comparison.second)
                mono=true
            end
        end
        if(!mono)
            return false
        end
    end
    return true
end





is_monotone (generic function with 1 method)